In [1]:
from pyLensLib.piemd import piemd
from pyLensLib.pointsrc import pointsrc
from pyLensLib.sersic import *
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from skimage import measure
import shapely.geometry
from pyLensLib.critcau import CriticalLine
from shapely.ops import polygonize, unary_union
import astropy.io.fits as pyfits
from tqdm import tqdm
from pyLensLib.deflector import deflector
import pandas as pd
import skimage
import matplotlib.pyplot as plt
from pyLensLib.observation import observation
from pyLensLib.maputils import image_fit
from scipy.ndimage import map_coordinates
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord, funcs
from astropy.nddata import Cutout2D
from skimage.transform import rescale
from pyLensLib.maputils import image_fit, map_obj, contour_fit
import pymupds

# set up cosmology
co = FlatLambdaCDM(H0=70.0, Om0=0.3)

In [2]:
def rescale(image, newpix):
    image_rescaled = skimage.transform.rescale(image, newpix/image.shape[0])
    return image_rescaled

def cutout_image(image, coords, fov=200.0):
    
    hdul=image
    w = WCS(image)
    image_ = hdul.data
    size = u.Quantity((fov,fov), u.arcsec)
    position_ = SkyCoord(coords, frame = 'fk5', unit = 'deg')
    cutout_ = Cutout2D(image_, position_, size, wcs=w)
    w_up = cutout_.wcs
    header_ = w_up.to_header()
    hdu_hst = pyfits.PrimaryHDU(cutout_.data, header=header_)
    hdu_hst = formatNewHeader(hdu_hst, fov)
    
    return hdu_hst

In [3]:
def formatHeaderForCut(a_x, a_y, pix):
    
    a_x.header["CRPIX1"]  =  (a_x.shape[0]//2, "Pixel coordinate of reference point")            
    a_x.header["CRPIX2"]  =  (a_x.shape[0]//2, "Pixel coordinate of reference point")  
    a_x.header["CRVAL1"]  =  (0.0, "[deg] Coordinate value at reference point")      
    a_x.header["CRVAL2"]  =  (0.0, "[deg] Coordinate value at reference point")
    a_x.header["CDELT1"]  =  ( pix , "[deg] Coordinate increment at reference point")  
    a_x.header["CDELT2"]  =  ( pix , "[deg] Coordinate increment at reference point")  
    a_x.header["CUNIT1"]  =  ("deg", "Units of coordinate increment and value")        
    a_x.header["CUNIT2"]  =  ("deg", "Units of coordinate increment and value")        
    a_x.header["CTYPE1"]  =  ("RA---TAN", "Right ascension, gnomonic projection")          
    a_x.header["CTYPE2"]  =  ("DEC--TAN", "Declination, gnomonic projection") 
    
    
    a_y.header["CRPIX1"]  =  (a_y.shape[0]//2, "Pixel coordinate of reference point")            
    a_y.header["CRPIX2"]  =  (a_y.shape[0]//2, "Pixel coordinate of reference point")  
    a_y.header["CRVAL1"]  =  (0.0, "[deg] Coordinate value at reference point")      
    a_y.header["CRVAL2"]  =  (0.0, "[deg] Coordinate value at reference point")
    a_y.header["CDELT1"]  =  ( pix , "[deg] Coordinate increment at reference point")  
    a_y.header["CDELT2"]  =  ( pix , "[deg] Coordinate increment at reference point")  
    a_y.header["CUNIT1"]  =  ("deg", "Units of coordinate increment and value")        
    a_y.header["CUNIT2"]  =  ("deg", "Units of coordinate increment and value")        
    a_y.header["CTYPE1"]  =  ("RA---TAN", "Right ascension, gnomonic projection")          
    a_y.header["CTYPE2"]  =  ("DEC--TAN", "Declination, gnomonic projection")
    
    a_x.header["RADESYS"] = "FK5"
    a_y.header["RADESYS"] = "FK5"
    
    return a_x, a_y

def formatNewHeader(hdu, fov):
   
    hdu.header["XMAX"] = (fov/2, "xMax arcsec")
    hdu.header["XMIN"] = (-fov/2, "xMin arcsec")
    hdu.header["YMAX"] = (fov/2, "yMax arcsec")
    hdu.header["YMIN"] = (-fov/2, "yMin arcsec")
    hdu.header["CRPIX1"] = (hdu.shape[0]//2, "Pixel coordinate of reference point")
    hdu.header["CRPIX2"] = (hdu.shape[1]//2, "Pixel coordinate of reference point")
    
    del hdu.header["CRVAL1"]
    del hdu.header["CRVAL2"]
    
    return hdu

In [4]:
def formatHeaderDef(newAng, oldAng):
    
    hdu = pyfits.PrimaryHDU(newAng)
    hdu.header["XMIN"] = (oldAng.header["XMIN"], "xMin arcsec")
    hdu.header["XMAX"] = (oldAng.header["XMAX"], "xMax arcsec")
    hdu.header["YMIN"] = (oldAng.header["YMIN"], "yMin arcsec")
    hdu.header["YMAX"] = (oldAng.header["YMAX"], "yMax arcsec")
    
    return hdu

def selImage(file_sim, mu_image, lens, cnt):
    
    x, y = lens.getCritPoints(cnt, pixel_units=True)
    xmin, xmax, ymin, ymax = np.min(x), np.max(x), np.min(y), np.max(y)
    lim = max(int(round(xmax-xmin)), int(round(ymax-ymin)))
    xc, yc = (xmax-xmin)/2+xmin, (ymax-ymin)/2+ymin

    cutSim = file_sim[int(round(yc-lim/2-1)):int(round(yc+lim/2+1)),int(round(xc-lim/2-1)):int(round(xc+lim/2+1))]
    cutMap = mu_image[int(round(yc-lim/2-1)):int(round(yc+lim/2+1)),int(round(xc-lim/2-1)):int(round(xc+lim/2+1))]
    
    return cutSim, cutMap, xc, yc

def check(cnt, lens, point):
    
    x, y = lens.getCritPoints(cnt, pixel_units=True)
    xmin, xmax, ymin, ymax = np.min(x), np.max(x), np.min(y), np.max(y)
    i = []
    for p in point:
        if p[0] >= xmin and p[0] <=xmax and p[1] >= ymin and p[1] <= ymax:
            i.append([p[0],p[1]])
    i = np.array(i)
    
    return i

In [5]:
def multiple_images(se, noise_level, mu, mus, mask_unlensed, s, sFoldImages, xpos, ypos, pix, sz, model, lens, cntList, point):
    
    for i in range(len(cntList)):
        
        cutSim, cutMap, xc, yc = selImage(se.image, mu, lens, cntList[i])
        
        ai = image_fit(cutSim, ith= noise_level)
        cen = ai.centroid()
            
        #mu_point = cutMap[int(cen[1]),int(cen[0])]
        mask_lensed = cutSim > noise_level
        
        #ff, aa = plt.subplots(1,2,figsize=(10,10))
        #aa[0].imshow(cutSim, origin="lower", vmin=10e-6, vmax=10e-2, cmap=plt.cm.bone)
        #aa[1].imshow(cutMap, origin="lower", cmap=plt.cm.cubehelix)
        
        ratio_area = mask_lensed.sum()/mask_unlensed.sum()
        ratio_flux = np.sum(cutSim*mask_lensed)/np.sum(se.image_unlensed*mask_unlensed)
        mean_map = np.mean(cutMap[mask_lensed])
        median_map = np.median(cutMap[mask_lensed])
        weAvarage_map = np.average(cutMap[mask_lensed], weights=cutSim[mask_lensed])
        #point_map = mu_point
        
        mu_flux = np.sum((mus * se.image_unlensed) * mask_unlensed)
        flux = np.sum(se.image_unlensed * mask_unlensed)
        mu_true = mu_flux / flux
        
        #aa[0].plot(cen[0], cen[1], "o", markersize=1, color="red")
        #aa[1].plot(cen[0], cen[1], "o", markersize=1, color="red")
        
        pntInImage = check(cntList[i], lens, point)
        if len(pntInImage)==0:
            xi, yi = xc*pix-sz/2.0, yc*pix-sz/2.0
            point_map = cutMap[int(cen[1]),int(cen[0])]
            
            s_entry = {'x': xpos,'y': ypos,
                       'xi': xi,'yi': yi,
                       'mus_true': mu_true, 'area_ratio': ratio_area,
                       'flux_ratio': ratio_flux, model+'_mean_mu': mean_map,
                       model+'_median_mu':  median_map, model+'_wfl_mu':weAvarage_map, model+'_mu_point': point_map}
            s.append(s_entry)
        else:
            for n in pntInImage:
                xi, yi = n[0]*pix-sz/2.0, n[1]*pix-sz/2.0
                point_map = mu[int(n[1]),int(n[0])]
            
                s_entry = {'x': xpos,'y': ypos,
                           'xi': xi,'yi': yi,
                           'mus_true': mu_true, 'area_ratio': ratio_area,
                           'flux_ratio': ratio_flux, model+'_mean_mu': mean_map,
                           model+'_median_mu':  median_map, model+'_wfl_mu':weAvarage_map, model+'_mu_point': point_map}
            
                s.append(s_entry)
        
        if len(cntList) == 3 and len(point) == 3:
            for n in pntInImage:
                xi, yi = n[0]*pix-sz/2.0, n[1]*pix-sz/2.0
                point_map = mu[int(n[1]),int(n[0])]
                s_fold_entry = {'x': xpos,'y': ypos,
                                'xi': xi,'yi': yi,
                                'mus_true': mu_true, 'area_ratio': ratio_area,
                                'flux_ratio': ratio_flux, model+'_mean_mu': mean_map,
                                model+'_median_mu':  median_map, model+'_wfl_mu':weAvarage_map, model+'_mu_point': point_map, 
                                'flux': np.sum(cutSim*mask_lensed)} 
                sFoldImages.append(s_fold_entry)

In [6]:
def sort_contours(contours):
    c_all = []
    j = 0
    for contour in contours:
        contour[:, 0], contour[:, 1] = contour[:, 1].copy(), contour[:, 0].copy()
        ls = shapely.geometry.LineString(contour)
        lr = shapely.geometry.LineString(ls.coords[:] + ls.coords[0:1])
        mls = unary_union(lr)
        mp = shapely.geometry.MultiPolygon(list(polygonize(mls)))
        c = CriticalLine(j, mp)
        c.setPoints(contour)
        A = 0.0
        for g in range(len(mp)):
            A += mp[g].area
        c.setArea(A)
        c_all.append(c)
        j += 1
    c_sorted = sorted(c_all, key=lambda x: x.getArea(), reverse=True)
    return (c_sorted)

In [ ]:
zl, zsnorm, zslist = 0.5, 2, [1, 2, 4, 6]

model = [] #name of magnification map models, if these name is
           #present in .fits magnification file name 

for zs in zslist:
    mappa_ = [pyfits.open("")[0]] #magnification map

    fov = []
    for k in range(len(mappa_)):
        f = mappa_[k].header["CDELT2"]*3600*(mappa_[k].header["NAXIS1"]-1)
        fov.append(f)
    fovCut = round(min(fov),2)

    ang_ = pyfits.open("") #deflection angle map
    angx_, angy_ = ang_[0], ang_[1]
    
    pix_ = ((angx_.header['XMAX']-angx_.header['XMIN'])/angx_.header['NAXIS1'])/3600
    angx_, angy_ = formatHeaderForCut(angx_, angy_, pix_)
    angx_, angy_ = cutout_image(angx_, ["0 0"], fov=fovCut), cutout_image(angy_, ["0 0"], fov=fovCut)
    angx, angy = rescale(angx_.data, 4080), rescale(angy_.data, 4080)
    angx = formatHeaderDef(angx, angx_)

    pix = ((angx.header['XMAX']-angx.header['XMIN'])/angx.header['NAXIS1'])
    npix = angx.shape[0]
    sz = pix*(npix)

    kwargs = {
        'zl': zl,
        'zs': zsnorm
    }

    lens = deflector(co=co, angx=angx.data, angy=angy.data, usePotential=False, **kwargs)
    theta = np.linspace(-sz/2,sz/2,npix)
    lens.setGrid(theta)
    lens.change_redshift(zs)
    tEin = lens.thetaE()
    detA = (1.0 - lens.ka) ** 2 - (lens.g1 ** 2 + lens.g2 ** 2)
    mu=np.abs(1.0/detA)

    ct, cr = lens.tancl(), lens.radcl()
    caut, caur = lens.getCaustics(ct), lens.getCaustics(cr)
    
    print("Source's redshift: "+str(zs)+" - - - -")
    print("Shape: %5i px" % angx.shape[0])
    print("Fov: %5.2f arcsec" % sz)
    print("Pixel: %5.2f arcsec" % pix)
    print("Einstein's radius: %5.2f arcsec" % tEin)
    print("- - - - - - - - - - - - - - - - - - - ")
    
    angx4pntSrc, angy4pntSrc = rescale(angx.data, 2040), rescale(angy.data, 2040)
    pix4pntSrc = 4080/2040*pix
    npix4pntSrc = 2040
    lens4pntSrc = deflector(co=co, angx=angx4pntSrc, angy=angy4pntSrc, usePotential=False, **kwargs)
    theta4pntSrc = np.linspace(-sz/2,sz/2,npix4pntSrc)
    lens4pntSrc.setGrid(theta4pntSrc)
    lens4pntSrc.change_redshift(zs)
    
    mappa = [rescale(cutout_image(mappa_[0], ["0 0"], fov=fovCut).data, 4080),
             rescale(cutout_image(mappa_[1], ["0 0"], fov=fovCut).data, 4080),
             rescale(cutout_image(mappa_[2], ["0 0"], fov=fovCut).data, 4080)]
    
    mag_source, mag_lens, mag_sky = 22.0, 18.0, 22.5
    zp = 23.9
    
    y1 = lens.theta1 - lens.a1 + 2 * lens.pixel_scale
    y2 = lens.theta2 - lens.a2 + 2 * lens.pixel_scale
    mus = pymupds.mupds_triangle(y1 - lens.theta1.min(), y2 - lens.theta2.min(), lens.pixel_scale, detA, nray=len(theta))

    ob = observation(size=sz, Npix=npix, zp=zp, texp=565 * 4, bkg=mag_sky)
    fl_gal, fl_lens, sky_counts = ob.mag2counts(mag_source), ob.mag2counts(mag_lens), ob.bkg_counts
    # define a grid of sources
    fov_zs = int(tEin)
    thetasx = np.linspace(-7-fov_zs/2.0,-7+fov_zs/2.0,50)
    thetasy = np.linspace(-fov_zs/2.0,+fov_zs/2.0,50)
    thetas1, thetas2 = np.meshgrid(thetasx,thetasy)

    thetas1=thetas1.flatten()
    thetas2=thetas2.flatten()
            
    s0, s1, s2 = [], [], []
    s0F, s1F, s2F = [], [], []
    listaS, listaSF = [s0,s1,s2], [s0F, s1F, s2F]
    re, sersInd = 0.7, 4.0 #capire come fare sorgenti più estese
    
    #for i in tqdm(range(0,1)):
    for i in tqdm(range(0,2300)):
    #for i in tqdm(range(len(thetas1))):
    
        kwargs_source_light = {
            'n': sersInd,  # sersic index
            're': re,  # effective radius
            'q': 1.0,  # axis ratio
            'pa': np.pi / 4.,  # position angle
            'ys1': thetas1[i],  # 3.8,  # position x (on the source plane)
            'ys2': thetas2[i],  # 3.8,  # position y (on the source plane)
            'flux': fl_gal,  # flux of the source (in counts/s)
            'zs': zs
        }
        
        kwargs_source_light_4pntSrc = {
            'ys1': thetas1[i],   
            'ys2': thetas2[i],   
            'flux': fl_gal,  
            'zs': zs
        }

        se = sersic(size=sz, Npix=npix, gl=lens, save_unlensed=True, **kwargs_source_light)
        pntSrc = pointsrc(size=sz, Npix=npix4pntSrc, gl=lens4pntSrc, **kwargs_source_light_4pntSrc)
        imgPosition = pntSrc.find_images()
        noise = ob.makeNoise(se.image)
        
        if np.size(imgPosition)!=3:
            pntPos = []
            for j in range(len(imgPosition)):
                pntPos.append([imgPosition[0][j], imgPosition[1][j]])
            pntPos = np.array(pntPos)
            pntPos = (pntPos/pix4pntSrc+2040/2)*2
        else:
            imgPosition = np.array(imgPosition)
            imgPosition = (imgPosition/pix4pntSrc+2040/2)*2
            pntPos = [[imgPosition[0][0], imgPosition[1][0]]]
            pntPos = np.array(pntPos)
        
        #SNR = 1.0
        SNR = 5.0
        #SNR = 10.0
        noise_level = 0.5 * (1.0 + np.sqrt(1.0 + 4.0 * ob.bkg_counts * ob.texp)) / ob.texp * SNR

        cnt = se.image_contours(level=noise_level)
        contours___ = measure.find_contours(se.image_unlensed, noise_level)
        cnt_source = sort_contours(contours___)
    
        mask_lensed = se.image > noise_level
        mask_unlensed = se.image_unlensed > noise_level
    
        if len(cnt)!=1:
            for j in range(len(model)):
                multiple_images(se=se, noise_level=noise_level, mu=mappa[j], mus=mus.T, mask_unlensed=mask_unlensed, s=listaS[j], sFoldImages=listaSF[j], xpos=thetas1[i], ypos=thetas2[i], pix=pix, sz=sz, model=model[j], lens=lens, cntList=cnt, point=pntPos)
                continue
            continue

        ai = image_fit(se.image, ith=noise_level)
        cen = ai.centroid()

        area_ratio = mask_lensed.sum() / mask_unlensed.sum()
        flux_ratio = np.sum(se.image * mask_lensed) / np.sum(se.image_unlensed * mask_unlensed)
    
        mu_flux = np.sum((mus.T * se.image_unlensed) * mask_unlensed)
        flux = np.sum(se.image_unlensed * mask_unlensed)
        mu_true = mu_flux / flux
                        
        for k in range(len(model)):
            mean_mu, median_mu, wfl_mu, mu_point = np.mean(mappa[k][mask_lensed]), np.median(mappa[k][mask_lensed]), np.average(mappa[k][mask_lensed], weights=se.image[mask_lensed]), mappa[k][int(pntPos[0,1]), int(pntPos[0,0])]
            s_entry = {'x': thetas1[i],'y': thetas2[i],
                       'xi':cen[0]*pix-sz/2.0,'yi':cen[1]*pix-sz/2.,
                       'mus_true': mu_true, 'area_ratio': area_ratio,
                       'flux_ratio': flux_ratio, model[k]+'_mean_mu': mean_mu,
                       model[k]+'_median_mu': median_mu, model[k]+'_wfl_mu': wfl_mu, model[k]+'_mu_point': mu_point}
            listaS[k].append(s_entry)
        

    for m in range(len(model)):
        sdf = pd.DataFrame(listaS[m])
        sdfF = pd.DataFrame(listaSF[m])
        sdf.to_csv("")
        sdfF.to_csv("")